In [7]:
# %pip install numpy tensorflow

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Attention, Concatenate

# 定义输入特征数和类别数
num_features = 30
num_classes = 10 

# 定义输入层
sequence_input = Input(shape=(None, num_features), dtype='float32')

# 双向LSTM层
lstm_out = Bidirectional(LSTM(64, return_sequences=True))(sequence_input)

# 注意力机制
attention = Attention()([lstm_out, lstm_out])
context_vector = Concatenate()([lstm_out, attention])

# 输出层
output = Dense(num_classes, activation='softmax')(context_vector)

# 定义模型
model = Model(inputs=sequence_input, outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 输出模型结构
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, 30)]           0         []                            
                                                                                                  
 bidirectional_1 (Bidirecti  (None, None, 128)            48640     ['input_2[0][0]']             
 onal)                                                                                            
                                                                                                  
 attention_1 (Attention)     (None, None, 128)            0         ['bidirectional_1[0][0]',     
                                                                     'bidirectional_1[0][0]']     
                                                                                            

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Attention, Concatenate, TimeDistributed

# 定义输入特征数和类别数
num_features = 30
num_classes = 10 

# 定义输入层
sequence_input = Input(shape=(None, num_features), dtype='float32')

# 双向LSTM层
lstm_out = Bidirectional(LSTM(64, return_sequences=True))(sequence_input)

# 注意力机制
attention = Attention()([lstm_out, lstm_out])

# 连接Bi-LSTM的输出和注意力的输出
context_vector = Concatenate(axis=-1)([lstm_out, attention])

# 全连接层
dense_output = TimeDistributed(Dense(num_classes))(context_vector)

# Softmax层
output = tf.keras.layers.Softmax()(dense_output)

# 定义模型
model = Model(inputs=sequence_input, outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 输出模型结构
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, None, 30)]           0         []                            
                                                                                                  
 bidirectional_2 (Bidirecti  (None, None, 128)            48640     ['input_3[0][0]']             
 onal)                                                                                            
                                                                                                  
 attention_2 (Attention)     (None, None, 128)            0         ['bidirectional_2[0][0]',     
                                                                     'bidirectional_2[0][0]']     
                                                                                            

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Attention, Concatenate, TimeDistributed, Lambda

# 定义输入特征数和类别数
num_features = 30 
num_classes = 10 

# 定义输入层
sequence_input = Input(shape=(None, num_features), dtype='float32')

# 双向LSTM层
lstm_out = Bidirectional(LSTM(64, return_sequences=True))(sequence_input)

# 注意力机制
attention = Attention()([lstm_out, lstm_out])

# 连接Bi-LSTM的输出和注意力的输出
context_vector = Concatenate(axis=-1)([lstm_out, attention])

# 全连接层 + Softmax 层
# 将加权求和结果输入到全连接层进行维度转换
dense_output = TimeDistributed(Dense(num_classes))(context_vector)

# 使用 softmax 函数归一化
output = tf.keras.layers.Softmax()(dense_output)

# 获取 t+1 时间步的输出
# t+1 时间步是输入的最后一个时间步，所以我们取最后一个时间步的预测
final_output = Lambda(lambda x: x[:, -1, :])(output)

# 定义模型
model = Model(inputs=sequence_input, outputs=final_output)

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 输出模型结构
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, None, 30)]           0         []                            
                                                                                                  
 bidirectional_3 (Bidirecti  (None, None, 128)            48640     ['input_4[0][0]']             
 onal)                                                                                            
                                                                                                  
 attention_3 (Attention)     (None, None, 128)            0         ['bidirectional_3[0][0]',     
                                                                     'bidirectional_3[0][0]']     
                                                                                            

### 测试

In [14]:
import numpy as np

# 定义输入特征数和类别数
num_features = 30 
num_classes = 10 

# 生成测试数据
# 生成一批序列，每个序列有不同的长度，每个时间步有num_features个特征
batch_size = 5
sequence_length = 15  # 假设每个序列的长度为15
X_test = np.random.random((batch_size, sequence_length, num_features))

print("Test data shape:", X_test.shape)
# print(X_test)

# 使用模型进行预测
predictions = model.predict(X_test)

# 打印预测结果
# print("Predictions shape:", predictions.shape)
print("Predictions:", predictions) # 活动概率分布 APD


[[[0.42447092 0.24251966 0.67116598 ... 0.88051141 0.04932014 0.16036292]
  [0.46959367 0.93995296 0.81272675 ... 0.11763351 0.2473363  0.48661646]
  [0.60997488 0.31920114 0.89980062 ... 0.50193722 0.36517503 0.17928614]
  ...
  [0.61680108 0.55919483 0.2541663  ... 0.69023282 0.53553673 0.89482918]
  [0.28380233 0.50723815 0.7479413  ... 0.76256672 0.0172974  0.13241657]
  [0.14858227 0.04735218 0.33282684 ... 0.63022875 0.11150046 0.40315685]]

 [[0.81350712 0.05545562 0.71089439 ... 0.284262   0.56763787 0.98231231]
  [0.64421447 0.85206319 0.96900237 ... 0.60178744 0.23497083 0.47474367]
  [0.62308567 0.18881142 0.4013341  ... 0.5295352  0.30591888 0.72310612]
  ...
  [0.58090372 0.81816376 0.63868628 ... 0.67712883 0.09745508 0.99328394]
  [0.5053864  0.18124221 0.91120007 ... 0.78043068 0.30582116 0.53904417]
  [0.35974551 0.56960027 0.38458789 ... 0.19228921 0.05731467 0.51107426]]

 [[0.16754048 0.30241849 0.62300648 ... 0.01542437 0.07918195 0.37983646]
  [0.65979025 0.312486

### 详细步骤说明

1. **输入层**：我们假设输入数据已经被预处理为形状为 `(batch_size, sequence_length, num_features)` 的张量，其中 `num_features` 是每个时间步的特征数。

2. **Bi-LSTM层**：使用 `Bidirectional` 包装 `LSTM` 层，并设置 `return_sequences=True` 以确保输出每个时间步的输出。

3. **注意力层**：引入注意力机制，通过计算每个时间步的隐态输出之间的相似度系数得到注意力权重，然后与隐态向量加权求和得到注意力上下文。

4. **输出层**：通过全连接层和 Softmax 函数，将模型输出转换为下一个时间步的活动概率分布。